# Projet maison n° 3

In [1]:
# imports
import pandas as pd
from os import walk
import re
import numpy as np

## 1. US baby names

On va s'intéresser au dataset **National data** de la SSA : https://www.ssa.gov/oact/babynames/limits.html

1. Télécharger le dataset des prénoms US : https://www.ssa.gov/oact/babynames/names.zip

Lire la documentation associée.

2. Implémenter une fonction Python qui produit un unique DataFrame avec tous les fichiers en utilisant pandas (par ex. fonction "concat" ou méthode "append"), pas de bash :)

Ordre et noms des colonnes : 'year', 'name', 'gender', 'births'

Le DataFrame doit être trié selon l'année croissante puis selon l'ordre défini par les différents fichiers (voir la documentation du dataset).

In [16]:
def df_names_us():
    
    col_names = ['year','name','gender','births']
    df  = pd.DataFrame(columns=col_names)

    for year in range(1880,2020+1):
        file_name = f'names/yob{year}.txt'
        
        df_temp = pd.read_csv(file_name,sep=',',names=['name','gender','births'])
        df_temp['year'] =  year
        
        df = df.append(df_temp)

    df = df[col_names]

    df.year = df.year.astype(int)
    df.births = df.births.astype(int)

    df = df.reset_index(drop=True)

    return df

df_names_us().head()

,year,name,gender,births
0,1880,Mary,F,7065
1,1880,Anna,F,2604
2,1880,Emma,F,2003
3,1880,Elizabeth,F,1939
4,1880,Minnie,F,1746


## 2. Prénoms français

On va s'intéresser au dataset **Fichiers France hors Mayotte** de l'INSEE :  https://www.insee.fr/fr/statistiques/2540004/

L'idée est de charger les données et ensuite de les conformer au DataFrame des prénoms US. Ainsi, toute manipulation sur le DataFrame des prénoms US pourra être directement réutilisée avec le DataFrame des prénoms français.
 
1. Télécharger le dataset des prénoms français : https://www.insee.fr/fr/statistiques/fichier/2540004/nat2020_csv.zip


Lire la documentation, ça peut être utile...
 
2. Implémenter une fonction Python qui produit un DataFrame avec les prénoms français en prenant le DataFrame des prénoms US comme modèle :
 
 - Même ordre et mêmes noms des colonnes : year, name, gender, births
 - Mêmes dtypes pour les colonnes
 - Mêmes valeurs pour la colonne 'gender'
 - Seuls les prénoms de 2 caractères et plus sont conservés
 - La casse des prénoms doit être identique : initiales en majuscule, autres lettres en minuscule
 - Les lignes avec des données inutilisables doivent être supprimées
 - Les données sont triées à l'identique : années (↑), puis gender (↑), puis births (↓) et enfin name (↑)
 - L'index du DataFrame doit aller de 0 à N-1

In [9]:
def df_names_fr():
    file_name = 'nat2020.csv'
    col_names = ['year','name','gender','births']

    df = pd.read_csv(file_name,sep=';',
                        names=['gender','name','year','births'],
                        header=0,index_col=False,
                        dtype={'gender': 'int', 'name': 'string', 'births': 'int'})

    # reorder columns
    df = df[col_names]

    #clean
    df= df[df['year'] != 'XXXX']
    df= df[df['name'] != '_PRENOMS_RARES']
    df= df[df['name'].apply(len)>=2]

    # change type
    df['births'] = df['births'].astype('object')

    # transform gender
    df['gender'] = df['gender'].replace((1,2),('M','F'))

    # transform names
    df['name'] = df['name'].apply(lambda x: x[0]+str.lower(x[1:]))

    # reorder values
    df = df.sort_values(
        by=['year','gender','births','name'],
        ascending=[True,True,False,True])

    # reset index
    df = df.reset_index(drop=True)

    return df

df_names_fr().head()

,year,name,gender,births
0,1900,Marie,F,48713
1,1900,Jeanne,F,13981
2,1900,Marguerite,F,8058
3,1900,Germaine,F,6981
4,1900,Louise,F,6696


## 3. Taux de change

On va s'intéresser au dataset des cours des devises de la Banque de France :  http://webstat.banque-france.fr/fr/#/downloading

L'idée est de charger les données, de les nettoyer et de pouvoir accéder aux cours de certaines devises à partir de leur code ISO3.
 
1. Utiliser le dataset des taux de change fourni.


2. Implémenter une fonction qui produit un DataFrame avec les taux de change par date pour une liste de codes ISO3 de devises passée en argument. L'index du DataFrame doit correspondre aux dates (voir la fonction pd.to_datetime() avec le format '%d/%m/%Y'). Les colonnes du DataFrame doivent correspondre aux devises.

In [26]:
# taux de change
def df_taux_change(devises):
    file_path = 'Webstat_Export_20211004.csv'
    df_temp = pd.read_csv(file_path,sep=';',decimal=',',skiprows=[1,2,3,4,5])

    #create the reg_expr
    reg_expr = 'Titre'
    for devise in devises : reg_expr +='|'+ devise
    df = df_temp.filter(regex=(reg_expr))

    df = df.rename(columns={'Titre :':'Date'})
    df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
    df = df.set_index('Date')
    df = df.set_axis(devises,axis=1, inplace=False)
    
    #clean and change type
    df = df[df!='-'].dropna()
    df = df.replace(',','.',regex=True).astype('float')
    return df


### Tests

In [27]:
import unittest

class Lesson4Tests(unittest.TestCase):
    def test_df_names_us(self):
        df = df_names_us()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 2020863)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)
        
    def test_df_names_fr(self):
        df = df_names_fr()
        # colonnes
        self.assertEqual(list(df.columns), ['year', 'name', 'gender', 'births'])
        # lignes
        self.assertEqual(len(df), 630406)
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.range.RangeIndex))
        # test names
        self.assertTrue(df.loc[df['name'].str.contains('^[A-Z]+(?:-[A-Z]+)?$')].empty)
        # test gender
        self.assertEqual(len(df), len(df.loc[df['gender']=='F']) + len(df.loc[df['gender']=='M']))
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

    def test_df_taux_change(self):
        df = df_taux_change(['CHF', 'GBP', 'USD'])
        # colonnes
        self.assertEqual(list(df.columns), ['CHF', 'GBP', 'USD'])
        # index
        self.assertTrue(isinstance(df.index, pd.core.indexes.datetimes.DatetimeIndex))
        # types taux
        self.assertTrue((df.dtypes == 'float').all())
        # test NaN
        self.assertTrue(df.loc[df.isnull().any(axis=1)].empty)

In [28]:
# run tests
def run_tests():
    test_suite = unittest.makeSuite(Lesson4Tests)
    runner = unittest.TextTestRunner(verbosity=2)
    runner.run(test_suite)

In [29]:
# run tests

run_tests()

test_df_names_fr (__main__.Lesson4Tests) ... ok
test_df_names_us (__main__.Lesson4Tests) ... ok
test_df_taux_change (__main__.Lesson4Tests) ... ok

----------------------------------------------------------------------
Ran 3 tests in 16.926s

OK
